In [0]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
os.chdir('/content/drive/My Drive/scatnetgpu')
!pwd

In [0]:
!pip install Pillow
!pip install scipy==1.1.0
import pandas as pd
from pandas import ExcelWriter

from scipy.ndimage import imread

import skimage
import numpy as np
import pywt
import scipy.stats

import cv2

!pip install cpbd
import cpbd
from matplotlib import pyplot as plt
plt.ion()
import glob

from scipy import io 
import itertools
from scipy import interp

In [0]:
import skimage.feature
def effective_mean(patch):
    vec=patch.flatten()
    m1 = [i for i in vec if i!=0]
    return(np.mean(m1))

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray


def ReturnPixelStats(img):
    f = []
    #Pixel Statistics
    for channel in range(0,3):
        a = img[:,:,channel]
        vec = a.flatten()
        # 8 statistic characteristics of pixel values for each color
        f.append(np.mean(vec))
        f.append(np.var(vec))
        f.append(scipy.stats.skew(vec))
        f.append(scipy.stats.kurtosis(vec))
        f.append(np.percentile(vec,25))
        f.append(np.median(vec))
        f.append(np.percentile(vec,75))
        f.append(scipy.stats.entropy(vec))
    return f


def ReturnPixelStatsGrey(img):

    f = []
    vec = img
    #Pixel Statistics
    # 8 statistic characteristics of pixel values for each color
    f.append(np.mean(vec))
    f.append(np.var(vec))
    f.append(scipy.stats.skew(vec))
    f.append(scipy.stats.kurtosis(vec))
    f.append(np.percentile(vec,25))
    f.append(np.median(vec))
    f.append(np.percentile(vec,75))
    f.append(scipy.stats.entropy(vec))
    return f

def ExtractTexture(img):
    glcm = skimage.feature.greycomatrix(img,[1],[0, np.pi/4, np.pi/2, 3*np.pi/4],normed=True, symmetric=True)
    contrast = skimage.feature.greycoprops(glcm, prop = 'contrast')
    homogeneity = skimage.feature.greycoprops(glcm, prop = 'homogeneity')
    dissimilarity = skimage.feature.greycoprops(glcm, prop = 'dissimilarity')
    ASM = skimage.feature.greycoprops(glcm, prop = 'ASM')
    correlation = skimage.feature.greycoprops(glcm, prop = 'correlation')
    energy = skimage.feature.greycoprops(glcm, prop = 'energy')
    features = (np.array([contrast, homogeneity, dissimilarity, ASM, correlation, energy])).flatten()
    return features


def get_wavelet_features_gray(img):
    wp = pywt.WaveletPacket2D(data=img, wavelet='haar', mode='symmetric')
    feature=[]
    data=wp['h'].data
    vec=data.flatten()
    feature.append(ReturnPixelStatsGrey(vec))

    data=wp['v'].data
    vec=data.flatten()
    feature.append(ReturnPixelStatsGrey(vec))

    data=wp['d'].data
    vec=data.flatten()
    feature.append(ReturnPixelStatsGrey(vec))

    return feature




def ImageFeatures(img):
    (h,l) = img[:,:,0].shape
    #Initialize feature vector to 0
    f = []
    
    #Pixel Statistics
    f.extend(ReturnPixelStats(img))

    #Texture Features
    #Function Takes uint8 greyscale image as input
    grey_img = rgb2gray(img)
    a_8 = grey_img.astype(np.uint8)
    f.extend(ExtractTexture(a_8))
    
    #Extracting Region Specific Features
    a = grey_img
    A1 = a[:int(h/4),:int(l/4)]
    A2 = a[:int(h/4), int(5*l/24):int(5*l/24+l/4)]
    A3 = a[:int(h/4):,int(3*l/4):]
    A4 = a[int(5*h/24):int(5*h/24+h/4),:int(l/4)]
    A6 = a[int(5*h/24): int(5*h/24+h/4),int(l-l/4):]
    A7 = a[-int(h/4):,:int(l/4)]
    A8 = a[int(h-h/4):,int(5*l/24):int(5*l/24+l/4)]
    A9 = a[int(h-h/4):, int(l-l/4):]
    #Central Pixel Statistics
    A5_color = img[int(5*h/24):int(5*h/24+h/4), int(5*l/24):int(5*l/24+l/4),:]
    f.extend(ReturnPixelStats(A5_color))
    #features to define symmetricity of the fundus image
    m1=effective_mean(A1)
    m2=effective_mean(A2)
    m3=effective_mean(A3)
    m4=effective_mean(A4)
    m6=effective_mean(A6)
    m7=effective_mean(A7)
    m8=effective_mean(A8)
    m9=effective_mean(A9)
    d1 = m1-m9
    d2 = m2-m8
    d3 = m3-m7
    d4 = m4-m6
    f.append(d1)
    f.append(d2)
    f.append(d3)
    f.append(d4)
    
    #w=[]

    #wavelet features
    #w.extend(get_wavelet_features_gray(n[:,:,0]))
    #w.extend(get_wavelet_features_gray(n[:,:,1]))
    #w.extend(get_wavelet_features_gray(n[:,:,2]))
    #f.extend(w)

    
    
    

    #CPBD Blur Feature
    cp = cpbd.compute(grey_img)
    f.append(cp)
    return f


In [0]:
import cv2
import os
import pandas as pd

from skimage.color import rgb2hsv

import cv2
import numpy as np




import os
#os.chdir('/content/drive/My Drive/scatnetgpu/All_images')
os.chdir('/content/drive/My Drive/scatnetgpu/Onsite_all')
!pwd



#K = np.zeros((1200, 77),dtype=float)
K = np.zeros((400, 77),dtype=float)
cnt=0

for iter in range(1,101):
  filename = "{:03d}".format(iter) + '_' + str(1) +'.jpg'
  #filename = "{:03d}".format(iter) + '_l' + str(1) +'.jpg'
  isExist = os.path.exists(os.path.join("/content/drive/My Drive/scatnetgpu/Onsite_all/",filename)) 
  #isExist = os.path.exists(os.path.join("/content/drive/My Drive/scatnetgpu/All_images/",filename)) 
  
  if isExist:
    for i in range(1,5):
      if (i<3):
        filename = "{:03d}".format(iter) + '_' + str(i) +'.jpg'
        #filename = "{:03d}".format(iter) + '_l' + str(i) +'.jpg'
      else:
        #filename = "{:03d}".format(iter) + '_r' + str(i-2) +'.jpg'
        filename = "{:03d}".format(iter) + '_' + str(i-2) +'.jpg'
      
      Im = cv2.imread(os.path.join("/content/drive/My Drive/scatnetgpu/Onsite_all/",filename))
      #Im = cv2.imread(os.path.join("/content/drive/My Drive/scatnetgpu/All_images/",filename))

      # ----  polar coordinates  ------
      Im = Im.astype(np.float32)
      value = np.sqrt(((Im.shape[0]/2.0)**2.0)+((Im.shape[1]/2.0)**2.0))
      
      polar_image = cv2.linearPolar(Im,(Im.shape[0]/2, Im.shape[1]/2), value, cv2.WARP_FILL_OUTLIERS)
      polar_image = polar_image.astype(np.uint8)
      #from google.colab.patches import cv2_imshow

      img = polar_image
      
      (h,l) = img[:,:,0].shape
      #Initialize feature vector to 0
      f = []
      #Pixel Statistics
      f.extend(ReturnPixelStats(img))
      #Texture Features
      #Function Takes uint8 greyscale image as input
      grey_img = rgb2gray(img)
      a_8 = grey_img.astype(np.uint8)
      f.extend(ExtractTexture(a_8))

      #Extracting Region Specific Features
      a = grey_img
      A1 = a[:int(h/4),:int(l/4)]
      A2 = a[:int(h/4), int(5*l/24):int(5*l/24+l/4)]
      A3 = a[:int(h/4):,int(3*l/4):]
      A4 = a[int(5*h/24):int(5*h/24+h/4),:int(l/4)]
      A6 = a[int(5*h/24): int(5*h/24+h/4),int(l-l/4):]
      A7 = a[-int(h/4):,:int(l/4)]
      A8 = a[int(h-h/4):,int(5*l/24):int(5*l/24+l/4)]
      A9 = a[int(h-h/4):, int(l-l/4):]
      #Central Pixel Statistics
      A5_color = img[int(5*h/24):int(5*h/24+h/4), int(5*l/24):int(5*l/24+l/4),:]
      f.extend(ReturnPixelStats(A5_color))
      #features to define symmetricity of the fundus image
      m1=effective_mean(A1)
      m2=effective_mean(A2)
      m3=effective_mean(A3)
      m4=effective_mean(A4)
      m6=effective_mean(A6)
      m7=effective_mean(A7)
      m8=effective_mean(A8)
      m9=effective_mean(A9)
      d1 = m1-m9
      d2 = m2-m8
      d3 = m3-m7
      d4 = m4-m6
      f.append(d1)
      f.append(d2)
      f.append(d3)
      f.append(d4)
      #CPBD Blur Feature
      cp = cpbd.compute(grey_img)
      f.append(cp)

      
      K[cnt,:] = f
      cnt = cnt+1
      print(cnt)
      
      #if cnt in range(100,1200,200):
TradFeat = K
TradFeat = pd.DataFrame(TradFeat)
import pandas as pd
from pandas import ExcelWriter


wr = ExcelWriter('On_Trad_feats_apr_22_polar.xlsx')
TradFeat.to_excel(wr,'sheet1')
wr.save()

